<a href="https://colab.research.google.com/github/KGL8/Code-for-Lab/blob/main/photon_through_deuteron_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
from pythreejs import *
import ipywidgets as widgets

def vector_from_angles(theta, phi):
    return np.array([np.sin(theta), np.cos(theta) * np.sin(phi), np.cos(theta) * np.cos(phi)])

def intersect_ray_box(ray_origin, ray_direction, box_min, box_max):
    pass

focal_point_distance = 3
width = 2
height = 5
depth = 7
box_min = np.array([-width / 2, -height / 2, focal_point_distance])
box_max = np.array([width / 2, height / 2, depth + focal_point_distance])
ray_direction = vector_from_angles(0.5,0.5)

In [40]:
box_geometry = BoxBufferGeometry(width=width, height=height, depth=depth)
box_material = MeshBasicMaterial(color='blue', wireframe=True)
box = Mesh(geometry=box_geometry, material=box_material)
box.position = [0, 0, depth / 2 + focal_point_distance]

ray_line = Line(
    geometry=BufferGeometry(
        attributes={
            'position': BufferAttribute(array=np.array([np.array([0,0,0]), 100 * np.array(ray_direction)]), normalized=False),
        }
    ),
    material=LineBasicMaterial(color='red')
)

points = Points(
    geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute([[0,0,0]], normalized=False)
        }
    ),
    material = PointsMaterial(color="green", size=5, sizeAttenuation=False)
)

ambient_light = AmbientLight(color='white', intensity=0.2)

scene = Scene(children=[box, ray_line, points, ambient_light])
camera = PerspectiveCamera(position=[10, 10, 10], up=[0, 1, 0], children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=800, height=600)

display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_5e9102d0-9c98-4257-9005-088d1ef60a70'.

In [12]:
from google.colab import output
output.enable_custom_widget_manager()